# Documents builder

Creates json documents from .txt files that has the next jerarquy:

```
hierarchy = {
    'TITULO' : 'headline',
    'DISPOSICIONES' : 'headline',
    'CAPITULO' : 'chapter',
    'ARTÍCULO' : 'article'

}
```

In [1]:
from classifier import *

In [2]:
with open('constitucion_colombiana.txt', 'r') as f:
    text = f.readlines()

    f.close()
    
for line in text:
    if line == '\n':
        text.remove(line)

In [3]:
clean_text = text_remove_nl(text)
# Drop the title "Constitucion Politica de Colombia"
clean_text = clean_text[1:len(text)]

In [4]:
book_id = "Const"
art_list = articles_info(book_id, clean_text, debugging=False)
print('total articles = ', len(art_list))

total articles =  439


In [5]:
art_list[78]

{'index': 'constitucion_politica_de_colombia',
 'id': 'const02003069',
 'headline': {'title': 'TITULO II',
  'name': 'DE LOS DERECHOS, LAS GARANTIAS Y LOS DEBERES',
  'headline_id': 2},
 'chapter': {'title': 'CAPITULO 3',
  'name': 'DE LOS DERECHOS COLECTIVOS Y DEL AMBIENTE',
  'chapter_id': 3},
 'article': {'name': 'Artículo 79',
  'content': ['Artículo 79. Todas las personas tienen derecho a gozar de un ambiente sano. La ley garantizará la participación de la comunidad en las decisiones que puedan afectarlo.',
   'Es deber del Estado proteger la diversidad e integridad del ambiente, conservar las áreas de especial importancia ecológica y fomentar la educación para el logro de estos fines.'],
  'article_id': 'Const02003069',
  'lexical_diversity': 10.147058823529411}}

In [6]:
first=77
last_p1=first + 1
for n in range(len(art_list[first:last_p1])):
    art = n+first
    print(f'Article No {art+1}: \n',
          "id: ", art_list[art]['article']['article_id'],"\n",
          "lexical_diversity: ", art_list[art]['article']['lexical_diversity'],"\n",
          art_list[art]['headline']['title'],art_list[art]['headline']['name'],"\n",
          art_list[art]['chapter']['title'],art_list[art]['chapter']['name'],"\n",
          art_list[art]['article']['name'],art_list[art]['article']['content'])
    print('\n--------------------------------------\n')

Article No 78: 
 id:  Const02003068 
 lexical_diversity:  18.583333333333332 
 TITULO II DE LOS DERECHOS, LAS GARANTIAS Y LOS DEBERES 
 CAPITULO 3 DE LOS DERECHOS COLECTIVOS Y DEL AMBIENTE 
 Artículo 78 ['Artículo 78. La ley regulará el control de calidad de bienes y servicios ofrecidos y prestados a la comunidad, así como la información que debe suministrarse al público en su comercialización.', 'Serán responsables, de acuerdo con la ley, quienes en la producción y en la comercialización de bienes y servicios, atenten contra la salud, la seguridad y el adecuado aprovisionamiento a consumidores y usuarios. El Estado garantizara la participación de las organizaciones de consumidores y usuarios en el estudio de las disposiciones que les conciernen. Para gozar de este derecho las organizaciones deben ser representativas y observar procedimientos democráticos internos.']

--------------------------------------



In [7]:
json_list = format_articles(art_list, debugging=False)


In [8]:
dict_json = json.dumps(json_list, ensure_ascii=False)
file = open('es_col_constitution.json', "w")
file.write(dict_json)
file.close()

In [9]:
article = json_list[0]
for key, value in article.items():
    print(key, ':', value)

index : constitucion_politica_de_colombia
id : const01000001
headline : {'title': 'TITULO I', 'name': 'DE LOS PRINCIPIOS FUNDAMENTALES'}
chapter : {'title': None, 'name': None}
article : {'name': 'Artículo 1', 'content': ['Artículo 1. Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista, fundada en el respeto de la dignidad humana, en el trabajo y la solidaridad de las personas que la integran y en la prevalencia del interés general.']}


In [10]:
json_list[1]

{'index': 'constitucion_politica_de_colombia',
 'id': 'const01000002',
 'headline': {'title': 'TITULO I', 'name': 'DE LOS PRINCIPIOS FUNDAMENTALES'},
 'chapter': {'title': None, 'name': None},
 'article': {'name': 'Artículo 2',
  'content': ['Artículo 2. Son fines esenciales del Estado: servir a la comunidad, promover la prosperidad general y garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución; facilitar la participación de todos en las decisiones que los afectan y en la vida económica, política, administrativa y cultural de la Nación; defender la independencia nacional, mantener la integridad territorial y asegurar la convivencia pacifica y la vigencia de un orden justo.',
   'Las autoridades de la República están instituidas para proteger a todas las personas residentes en Colombia, en su vida, honra, bienes, creencias, y demás derechos y libertades, y para asegurar el cumplimiento de los deberes sociales del Estado y de los particulares.']}

In [12]:
import requests
import json

In [36]:
post_payload = json_list[1]

api_uri_prod = "https://searcher-col.herokuapp.com"
main_url = f"{api_uri_prod}/api/es/search"

load_test = requests.post(f"{api_uri_prod}/api/es/constitucion", json=post_payload)

print(load_test.json())
print(load_test.status_code)

{'statusCode': 400, 'error': 'Bad Request', 'message': '"subhead" is required'}
400


In [32]:
load_dict = json.loads(load_test.text)
print(load_dict['statusCode'])
print(load_dict['message'])
print(load_dict['error'])

400
"subhead" is required
Bad Request


In [34]:
error_string = '{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}'
error_500  = json.loads(error_string)
print(error_500['statusCode'])
print(error_500['message'])
print(error_500['error'])

500
An internal server error occurred
Internal Server Error


In [35]:
error_string = '{"statusCode":400,"error":"Bad Request","message":"\\"headline\\" must be of type object"}'
error_400  = json.loads(error_string)
print(error_400['statusCode'])
print(error_400['message'])
print(error_400['error'])

400
"headline" must be of type object
Bad Request


In [11]:
import requests
import json

es_url = 'https://searcher-col.herokuapp.com/api/es/constitucion'

art_test = json_list[12]
header = {'Content-Type': 'application/json'}
# r = requests.post(es_url, json= art_test)
# r.text

In [19]:
# art_test = json_list[13]
# r = requests.post(es_url, data= art_test)
r.text

'{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}'

In [20]:
api_uri_prod = "https://searcher-col.herokuapp.com"
main_url = f"{api_uri_prod}/api/es/search"

for article in json_list[11]:
    r = requests.post(main_url, json=article)
    print(r.text)
    

{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}
{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}
{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}
{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}
{"statusCode":500,"error":"Internal Server Error","message":"An internal server error occurred"}


In [14]:
test = requests.get(es_url, json=)

NameError: name 'r' is not defined

In [ ]:
# for art in range(len(art_list)):
#    lex_div = lexical_diversity(art_list[art]['article']['content'])
#    art_list[art]['article']['lexical_diversity'] = lex_div
#    print(art_list[art]['article'])

# ES Formated

from elasticsearch import Elasticsearch
from elasticsearch import helpers

es_url = 'https://searcher-col.herokuapp.com/api/es/constitucion'
es = Elasticsearch([es_url])

r = es.get(index=["const02003069"])
r

# Prueba
test_text = ["Esto, solo* es. un! texto{ de } prueba&", '*2']
test_text = clean_words(test_text, debugging=True)
print(test_text)

titles_info = titles_content(art_list)
titles_info


In [ ]:
art_test